In [ ]:
!pip install wandb 

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import  Conv2D,MaxPooling2D,Dropout,Flatten,Dense
from tensorflow.keras import Model

from tqdm import tqdm

import wandb
from wandb.keras import WandbCallback

import numpy as np
import cv2

In [ ]:
wandb.init(project="Face Recognition")

# **Load Dataset**

In [ ]:
dataset_path = "/content/drive/MyDrive/Datasets/7-7 dataset"

width = height = 224

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    zoom_range = 0.1,
    brightness_range =  (0.9, 1.1), 
    validation_split = 0.2
)

train_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'training'
)

val_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'validation'
)

# **Model**

In [14]:
batch_size = 32
epochs = 7

In [15]:
class MahdisNet(Model):
  def __init__(self):
    super().__init__()

    self.Conv2D_1 = Conv2D(32, (3,3), activation = 'relu',input_shape=(width, height, 3))
    self.Conv2D_2 = Conv2D(64, (3,3), activation = 'relu')
    self.MaxPooling = MaxPooling2D()
    self.flatten = Flatten()
    self.dense_1 = Dense(128, activation = 'relu')
    self.dense_2 = Dense(14, activation='softmax')
    self.dropout = Dropout(0.5)

  def call(self, x):
    y = self.Conv2D_1(x)
    z = self.MaxPooling(y)
    j = self.Conv2D_2(z)
    k = self.MaxPooling(j)
    m = self.flatten(k)
    n = self.dense_1(m)
    w = self.dropout(n)
    out = self.dense_2(w)

    return out 

In [16]:
model = MahdisNet()

In [17]:
config=wandb.config
config.learning_rate=0.001

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

loss_function = tf.keras.losses.CategoricalCrossentropy()

In [19]:
train_loss = tf.keras.metrics.MeanAbsoluteError()
test_loss = tf.keras.metrics.MeanAbsoluteError()
train_accuracy = tf.keras.metrics.CategoricalAccuracy()
test_accuracy = tf.keras.metrics.CategoricalAccuracy()

In [20]:
#train loop
#.fit()
for epoch in range (epochs):
  train_accuracy.reset_states()
  test_accuracy.reset_states()
  train_loss.reset_states()
  test_loss.reset_states()
  print("Epoch: ", epoch)

  #Training------------------------------
  for i ,(images, labels) in enumerate(tqdm(train_data)):
    if len(train_data) <= i:
            break
    with tf.GradientTape() as gTape:
      predictions = model(images)
      loss = loss_function(labels, predictions)

      train_loss(labels, predictions)
      train_accuracy(labels, predictions)

    #محاسبات مشتق ها
    gradiants = gTape.gradient(loss, model.trainable_variables)
    
    #update w
    optimizer.apply_gradients(zip(gradiants, model.trainable_variables))
  
  #Validation---------------------------
  for i, (images, labels) in enumerate(tqdm(val_data)):
    if len(train_data) <= i:
            break
    predictions = model(images)
    loss = loss_function(labels, predictions)
    test_accuracy(labels, predictions)
    test_loss(labels, predictions)

  print("Train Accuracy: ",train_accuracy.result())
  print("Test Accuracy: ",test_accuracy.result())
  print("Train loss : ", train_loss.result())
  print("Test loss : ", test_loss.result())

Epoch:  0


100%|██████████| 35/35 [00:44<00:00,  1.28s/it]
35it [00:32,  1.07it/s]


Train Accuracy:  tf.Tensor(0.18715596, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.15754718, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.12995344, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.120687366, shape=(), dtype=float32)
Epoch:  1


100%|██████████| 35/35 [00:35<00:00,  1.00s/it]
35it [00:33,  1.04it/s]


Train Accuracy:  tf.Tensor(0.5146789, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.4745283, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.09841359, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.086868405, shape=(), dtype=float32)
Epoch:  2


...

Train Accuracy:  tf.Tensor(0.89266056, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.7509434, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.028412102, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.04491377, shape=(), dtype=float32)
Epoch:  5


100%|██████████| 35/35 [00:33<00:00,  1.04it/s]
35it [00:32,  1.08it/s]


Train Accuracy:  tf.Tensor(0.9211009, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.8273585, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.018996436, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.032817688, shape=(), dtype=float32)
Epoch:  6


100%|██████████| 35/35 [00:33<00:00,  1.05it/s]

35it [00:32,  1.09it/s]

Train Accuracy:  tf.Tensor(0.95137614, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.8320755, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.013553871, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.033371676, shape=(), dtype=float32)
Train Accuracy:  tf.Tensor(0.95137614, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.8320755, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.013553871, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.033371676, shape=(), dtype=float32)


In [33]:
model.save_weights("/content/drive/MyDrive/Datasets/face_recognition.h5")